# Zillow Price Prediction

In this book, we have input  properties of houses like Longitude, Latitude, Area, Number of rooms etc. We also have the log error between the zestimate of the house and actual proce of the house. Log error is defined as 

logerror=log(Zestimate)−log(SalePrice)

We will use the properties to predict the log error using regression models.

In [54]:
#import needed libraries
import pandas as pd
import numpy as np
import sklearn
import seaborn as sns
import matplotlib.pyplot as plt

In [55]:
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor, ExtraTreesRegressor, GradientBoostingRegressor
from sklearn.svm import LinearSVR
from sklearn.linear_model import Lasso, ElasticNet
from sklearn.cross_validation import KFold;

In [56]:
train = pd.read_csv('properties_2016.csv')
train_label = pd.read_csv('train_2016_v2.csv')

c:\users\the hacker supreme\appdata\local\programs\python\python35\lib\site-packages\IPython\core\interactiveshell.py:2683: DtypeWarning: Columns (22,32,34,49,55) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [57]:
train.head()

,parcelid,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,bedroomcnt,buildingclasstypeid,buildingqualitytypeid,calculatedbathnbr,decktypeid,...,numberofstories,fireplaceflag,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,taxdelinquencyflag,taxdelinquencyyear,censustractandblock
0,10754147,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,9.0,2015.0,9.0,NaN,NaN,NaN,NaN
1,10759547,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,27516.0,2015.0,27516.0,NaN,NaN,NaN,NaN
2,10843547,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,650756.0,1413387.0,2015.0,762631.0,20800.37,NaN,NaN,NaN
3,10859147,NaN,NaN,NaN,0.0,0.0,3.0,7.0,NaN,NaN,...,1.0,NaN,571346.0,1156834.0,2015.0,585488.0,14557.57,NaN,NaN,NaN
4,10879947,NaN,NaN,NaN,0.0,0.0,4.0,NaN,NaN,NaN,...,NaN,NaN,193796.0,433491.0,2015.0,239695.0,5725.17,NaN,NaN,NaN


In [58]:
train.shape

(2985217, 58)

In [59]:
train_label.head()

,parcelid,logerror,transactiondate
0,11016594,0.0276,2016-01-01
1,14366692,-0.1684,2016-01-01
2,12098116,-0.0040,2016-01-01
3,12643413,0.0218,2016-01-02
4,14432541,-0.0050,2016-01-02


For model training, We only need the information on houses for which we have the logerror available. We therefore match the train table with train_label. 

In [60]:
merge = train.merge(train_label,on=['parcelid'])
merge.shape

(90275, 60)

In [61]:
merge.to_csv('merge.csv')

In [62]:
#divide into x and y
y_train = merge['logerror']
x_train = merge.drop(['logerror', 'parcelid'], axis=1)

We now clean the data. For this we do:
1. Remove duplicate columns
2. One hot encode text columns
3. Remove NAN values
4. Merge any columns as required

In [63]:
x_train = merge
x_train['year'], x_train['month'], x_train['day'] = x_train['transactiondate'].str.split('-').str
x_train = merge.drop(['transactiondate'], axis=1)

In [64]:
x_train.shape

(90275, 62)

In [65]:
#drop indistinct features
x_train=x_train.drop(['assessmentyear'], axis=1)
x_train.shape

(90275, 61)

In [66]:
#One hot encode boolean features
# OneHotEncoding
x_train['has_basement'] = x_train["basementsqft"].apply(lambda x: 0 if np.isnan(x) else 1).astype(float)
x_train['hashottuborspa'] = x_train["hashottuborspa"].apply(lambda x: 0 if np.isnan(x) else 1).astype(float)
x_train['has_pool'] = x_train["poolcnt"].apply(lambda x: 0 if np.isnan(x) else 1).astype(float)
x_train['has_airconditioning'] = x_train["airconditioningtypeid"].apply(lambda x: 0 if np.isnan(x) else 1).astype(float)


#handle missing values and nan
x_train['fireplacecnt'] = x_train['fireplacecnt'].apply(lambda x: 0 if np.isnan(x) else x).astype(float)
x_train['bathroomcnt'] = x_train['bathroomcnt'].fillna(x_train['bathroomcnt'].median()).astype(float)
x_train['bedroomcnt'] = x_train['bedroomcnt'].fillna(x_train['bedroomcnt'].median()).astype(float)
x_train['roomcnt'] = x_train['roomcnt'].fillna(x_train['roomcnt'].median()).astype(float)

x_train['taxamount'] = x_train['taxamount'].fillna(x_train['taxamount'].median()).astype(float)
x_train['landtaxvaluedollarcnt'] = x_train['landtaxvaluedollarcnt'].fillna(x_train['landtaxvaluedollarcnt'].median()).astype(float)
x_train['taxvaluedollarcnt'] = x_train['taxvaluedollarcnt'].fillna(x_train['taxvaluedollarcnt'].median()).astype(float)
x_train['structuretaxvaluedollarcnt'] = x_train['structuretaxvaluedollarcnt'].fillna(x_train['structuretaxvaluedollarcnt'].median()).astype(float)
x_train['garagetotalsqft'] = x_train['garagetotalsqft'].fillna(x_train['garagetotalsqft'].median()).astype(float)
x_train['garagecarcnt'] = x_train['garagecarcnt'].fillna(x_train['garagecarcnt'].median()).astype(float)
x_train['fireplacecnt'] = x_train['fireplacecnt'].fillna(x_train['fireplacecnt'].median()).astype(float)
x_train['calculatedfinishedsquarefeet'] = x_train['calculatedfinishedsquarefeet'].fillna(x_train['calculatedfinishedsquarefeet'].median()).astype(float)
x_train['yearbuilt'] = train['yearbuilt'].fillna(train['yearbuilt'].median()).astype(float)
x_train['lotsizesquarefeet'] = x_train['lotsizesquarefeet'].fillna(x_train['lotsizesquarefeet'].median()).astype(float)

x_train['longitude'] = x_train['longitude'].fillna(x_train['longitude'].median()).astype(float)
x_train['latitude'] = x_train['latitude'].fillna(x_train['latitude'].median()).astype(float)


In [67]:
x_train.head()

,parcelid,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,bedroomcnt,buildingclasstypeid,buildingqualitytypeid,calculatedbathnbr,decktypeid,...,taxdelinquencyflag,taxdelinquencyyear,censustractandblock,logerror,year,month,day,has_basement,has_pool,has_airconditioning
0,17073783,NaN,NaN,NaN,2.5,3.0,NaN,NaN,2.5,NaN,...,NaN,NaN,6.111002e+13,0.0953,2016,01,27,0.0,0.0,0.0
1,17088994,NaN,NaN,NaN,1.0,2.0,NaN,NaN,1.0,NaN,...,NaN,NaN,6.111002e+13,0.0198,2016,03,30,0.0,0.0,0.0
2,17100444,NaN,NaN,NaN,2.0,3.0,NaN,NaN,2.0,NaN,...,NaN,NaN,6.111001e+13,0.0060,2016,05,27,0.0,0.0,0.0
3,17102429,NaN,NaN,NaN,1.5,2.0,NaN,NaN,1.5,NaN,...,NaN,NaN,6.111001e+13,-0.0566,2016,06,07,0.0,0.0,0.0
4,17109604,NaN,NaN,NaN,2.5,4.0,NaN,NaN,2.5,NaN,...,NaN,NaN,6.111001e+13,0.0573,2016,08,08,0.0,0.0,0.0


In [68]:
#remove columns with insufficient data, duplicate columns
drop_columns = ['airconditioningtypeid','basementsqft','architecturalstyletypeid','buildingclasstypeid','decktypeid','fips', 'taxdelinquencyyear','censustractandblock','pooltypeid10','pooltypeid2','pooltypeid7','propertyzoningdesc','rawcensustractandblock','regionidcounty','regionidneighborhood','regionidzip','regionidcity','regionidneighborhood','regionidzip','storytypeid','threequarterbathnbr','typeconstructiontypeid','unitcnt','yardbuildingsqft17','yardbuildingsqft26','fireplaceflag','numberofstories','finishedfloor1squarefeet','calculatedbathnbr','fullbathcnt','propertycountylandusecode','propertylandusetypeid','taxvaluedollarcnt','taxdelinquencyflag','has_pool','calculatedbathnbr']
x_train=x_train.drop(drop_columns, axis=1)

In [69]:
x_train.shape

(90275, 31)

In [70]:
x_train.head()

,parcelid,bathroomcnt,bedroomcnt,buildingqualitytypeid,calculatedfinishedsquarefeet,finishedsquarefeet12,finishedsquarefeet13,finishedsquarefeet15,finishedsquarefeet50,finishedsquarefeet6,...,yearbuilt,structuretaxvaluedollarcnt,landtaxvaluedollarcnt,taxamount,logerror,year,month,day,has_basement,has_airconditioning
0,17073783,2.5,3.0,NaN,1264.0,1264.0,NaN,NaN,548.0,NaN,...,1963.0,115087.0,76724.0,2015.06,0.0953,2016,01,27,0.0,0.0
1,17088994,1.0,2.0,NaN,777.0,777.0,NaN,NaN,777.0,NaN,...,1963.0,143809.0,95870.0,2581.30,0.0198,2016,03,30,0.0,0.0
2,17100444,2.0,3.0,NaN,1101.0,1101.0,NaN,NaN,1101.0,NaN,...,1963.0,33619.0,14234.0,591.64,0.0060,2016,05,27,0.0,0.0
3,17102429,1.5,2.0,NaN,1554.0,1554.0,NaN,NaN,1554.0,NaN,...,1948.0,45609.0,17305.0,682.78,-0.0566,2016,06,07,0.0,0.0
4,17109604,2.5,4.0,NaN,2415.0,2415.0,NaN,NaN,1305.0,NaN,...,1947.0,277000.0,277000.0,5886.92,0.0573,2016,08,08,0.0,0.0


In [71]:
finifhed_square_feet = x_train.loc[:, 'finishedsquarefeet12':'finishedsquarefeet6']

In [72]:
finifhed_square_feet.head()

,finishedsquarefeet12,finishedsquarefeet13,finishedsquarefeet15,finishedsquarefeet50,finishedsquarefeet6
0,1264.0,NaN,NaN,548.0,NaN
1,777.0,NaN,NaN,777.0,NaN
2,1101.0,NaN,NaN,1101.0,NaN
3,1554.0,NaN,NaN,1554.0,NaN
4,2415.0,NaN,NaN,1305.0,NaN


In [73]:
a=finifhed_square_feet[["finishedsquarefeet12", "finishedsquarefeet13","finishedsquarefeet15","finishedsquarefeet50","finishedsquarefeet6"]].max(axis=1)

In [74]:
a.head()

0    1264.0
1     777.0
2    1101.0
3    1554.0
4    2415.0
dtype: float64

In [75]:
x_train['finished_sq_ft'] =a

In [76]:
x_train.head()

,parcelid,bathroomcnt,bedroomcnt,buildingqualitytypeid,calculatedfinishedsquarefeet,finishedsquarefeet12,finishedsquarefeet13,finishedsquarefeet15,finishedsquarefeet50,finishedsquarefeet6,...,structuretaxvaluedollarcnt,landtaxvaluedollarcnt,taxamount,logerror,year,month,day,has_basement,has_airconditioning,finished_sq_ft
0,17073783,2.5,3.0,NaN,1264.0,1264.0,NaN,NaN,548.0,NaN,...,115087.0,76724.0,2015.06,0.0953,2016,01,27,0.0,0.0,1264.0
1,17088994,1.0,2.0,NaN,777.0,777.0,NaN,NaN,777.0,NaN,...,143809.0,95870.0,2581.30,0.0198,2016,03,30,0.0,0.0,777.0
2,17100444,2.0,3.0,NaN,1101.0,1101.0,NaN,NaN,1101.0,NaN,...,33619.0,14234.0,591.64,0.0060,2016,05,27,0.0,0.0,1101.0
3,17102429,1.5,2.0,NaN,1554.0,1554.0,NaN,NaN,1554.0,NaN,...,45609.0,17305.0,682.78,-0.0566,2016,06,07,0.0,0.0,1554.0
4,17109604,2.5,4.0,NaN,2415.0,2415.0,NaN,NaN,1305.0,NaN,...,277000.0,277000.0,5886.92,0.0573,2016,08,08,0.0,0.0,2415.0


In [77]:
#remove the extra finished square feet columns
drop_columns = ['finishedsquarefeet12','finishedsquarefeet13','finishedsquarefeet15','finishedsquarefeet50','finishedsquarefeet6']
x_train=x_train.drop(drop_columns, axis=1)

In [78]:
x_train.head()

,parcelid,bathroomcnt,bedroomcnt,buildingqualitytypeid,calculatedfinishedsquarefeet,fireplacecnt,garagecarcnt,garagetotalsqft,hashottuborspa,heatingorsystemtypeid,...,structuretaxvaluedollarcnt,landtaxvaluedollarcnt,taxamount,logerror,year,month,day,has_basement,has_airconditioning,finished_sq_ft
0,17073783,2.5,3.0,NaN,1264.0,0.0,2.0,0.0,0.0,NaN,...,115087.0,76724.0,2015.06,0.0953,2016,01,27,0.0,0.0,1264.0
1,17088994,1.0,2.0,NaN,777.0,0.0,1.0,0.0,0.0,NaN,...,143809.0,95870.0,2581.30,0.0198,2016,03,30,0.0,0.0,777.0
2,17100444,2.0,3.0,NaN,1101.0,0.0,2.0,441.0,0.0,NaN,...,33619.0,14234.0,591.64,0.0060,2016,05,27,0.0,0.0,1101.0
3,17102429,1.5,2.0,NaN,1554.0,1.0,2.0,460.0,0.0,NaN,...,45609.0,17305.0,682.78,-0.0566,2016,06,07,0.0,0.0,1554.0
4,17109604,2.5,4.0,NaN,2415.0,1.0,2.0,665.0,0.0,NaN,...,277000.0,277000.0,5886.92,0.0573,2016,08,08,0.0,0.0,2415.0


In [79]:
x_train.shape

(90275, 27)

In [80]:
#entering zero for NAN values of certain columns
x_train['buildingqualitytypeid'] = x_train['buildingqualitytypeid'].apply(lambda x: 0 if np.isnan(x) else x).astype(float)
x_train['heatingorsystemtypeid'] = x_train['heatingorsystemtypeid'].apply(lambda x: 0 if np.isnan(x) else x).astype(float)
x_train['poolcnt'] = x_train['poolcnt'].apply(lambda x: 0 if np.isnan(x) else x).astype(float)
x_train['poolsizesum'] = x_train['poolsizesum'].apply(lambda x: 0 if np.isnan(x) else x).astype(float)



In [81]:
x_train.head()

,parcelid,bathroomcnt,bedroomcnt,buildingqualitytypeid,calculatedfinishedsquarefeet,fireplacecnt,garagecarcnt,garagetotalsqft,hashottuborspa,heatingorsystemtypeid,...,structuretaxvaluedollarcnt,landtaxvaluedollarcnt,taxamount,logerror,year,month,day,has_basement,has_airconditioning,finished_sq_ft
0,17073783,2.5,3.0,0.0,1264.0,0.0,2.0,0.0,0.0,0.0,...,115087.0,76724.0,2015.06,0.0953,2016,01,27,0.0,0.0,1264.0
1,17088994,1.0,2.0,0.0,777.0,0.0,1.0,0.0,0.0,0.0,...,143809.0,95870.0,2581.30,0.0198,2016,03,30,0.0,0.0,777.0
2,17100444,2.0,3.0,0.0,1101.0,0.0,2.0,441.0,0.0,0.0,...,33619.0,14234.0,591.64,0.0060,2016,05,27,0.0,0.0,1101.0
3,17102429,1.5,2.0,0.0,1554.0,1.0,2.0,460.0,0.0,0.0,...,45609.0,17305.0,682.78,-0.0566,2016,06,07,0.0,0.0,1554.0
4,17109604,2.5,4.0,0.0,2415.0,1.0,2.0,665.0,0.0,0.0,...,277000.0,277000.0,5886.92,0.0573,2016,08,08,0.0,0.0,2415.0


In [82]:
x_train=x_train.drop(['finished_sq_ft'], axis=1)
x_train=x_train.drop(['logerror'], axis=1)

In [83]:

x_train=x_train.drop(['parcelid'], axis=1)

We will now scale the data to ensure that each property is able to provide equal value to our learning algorithm.

In [84]:
x_train.to_csv('x_train.csv')
#scale the data
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)

Divide the training data into a train test split and try out various regression models. The regression models will be trained on the training set and then tested against ytest using Mean Absolute Error as error metric. 

In [85]:
from sklearn.model_selection import train_test_split
xtrain, xtest, ytrain, ytest = train_test_split(x_train,y_train,test_size=0.2, random_state=2)

In [86]:
#Check some base models
from sklearn.metrics import mean_absolute_error
rf = RandomForestRegressor(max_depth=10, random_state=1)
rf.fit(xtrain, ytrain)
rf_y=rf.predict(xtest)
mean_absolute_error(ytest,rf_y)

0.069351085439598781

In [106]:
ls = Lasso(alpha=1e-6, normalize=True)
ls.fit(xtrain,ytrain)
ls_y=ls.predict(xtest)
mean_absolute_error(ytest,ls_y)


0.068490505024560097

In [107]:
el = ElasticNet(alpha=1e-7,fit_intercept=True, normalize=True)
el.fit(xtrain,ytrain)
el_y=el.predict(xtest)
mean_absolute_error(ytest,el_y)

0.068525009349137841

In [111]:
#random forrest with ADA Boosing on Decision Tree
from sklearn.tree import DecisionTreeRegressor
rf_2 = AdaBoostRegressor(DecisionTreeRegressor(max_depth=100),
                          n_estimators=300, random_state=1)
rf_2.fit(xtrain, ytrain)
rf_2_y=rf_2.predict(xtest)
mean_absolute_error(ytest,rf_2_y)

0.067882480587604904

In [99]:
from sklearn.decomposition import PCA
pca = PCA(n_components=5)
pca.fit(x_train)
x_train_PCA=pca.transform(x_train)

In [100]:
xtrainpca, xtestpca, ytrainpca, ytestpca = train_test_split(x_train_PCA,y_train,test_size=0.2, random_state=2)

In [101]:
xtrainpca.shape

(72220, 5)

In [109]:
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor, ExtraTreesRegressor, GradientBoostingRegressor
gbr = GradientBoostingRegressor(n_estimators=50, max_depth= 25, min_samples_split=2, learning_rate=0.01)
gbr.fit(xtrainpca, ytrainpca)
gbr_y=gbr.predict(xtestpca)
mean_absolute_error(ytestpca,gbr_y)

0.072321443682408199

In [103]:
etr=ExtraTreesRegressor(n_estimators=50, max_features=5,random_state=1)
etr.fit(xtrainpca, ytrainpca)
etr_y=etr.predict(xtestpca)
mean_absolute_error(ytestpca,etr_y)

0.080256450623096096

In [95]:
el = ElasticNet(alpha=1e-6,fit_intercept=True, normalize=True)
el.fit(xtrainpca,ytrainpca)
el_y=el.predict(xtestpca)
mean_absolute_error(ytestpca,el_y)

0.068710807204532875

In [96]:
from sklearn.model_selection import cross_val_score
rf_2 = AdaBoostRegressor(DecisionTreeRegressor(max_depth=50),
                          n_estimators=100, random_state=1)
scores_rf_2 = cross_val_score(rf_2, x_train_PCA, y_train, cv=5, scoring='neg_mean_absolute_error')
scores_rf_2

array([-0.07035981, -0.06881327, -0.07131217, -0.07263973, -0.0699745 ])

In [97]:

scores_el = cross_val_score(el, x_train_PCA, y_train, cv=5, scoring='neg_mean_absolute_error')
scores_el

array([-0.06847512, -0.06698877, -0.0691936 , -0.07008986, -0.06760575])

In [98]:
#find feature importances
etr.fit(x_train,y_train)
importances = etr.feature_importances_
indices = np.argsort(importances)[::-1]
for f in range(x_train.shape[1]):
    print("%d. feature %d (%f)" % (f + 1, indices[f], importances[indices[f]]))

1. feature 16 (0.090096)
2. feature 18 (0.087925)
3. feature 3 (0.086977)
4. feature 17 (0.085112)
5. feature 15 (0.084691)
6. feature 11 (0.083601)
7. feature 9 (0.082374)
8. feature 10 (0.082177)
9. feature 21 (0.078745)
10. feature 20 (0.064370)
11. feature 1 (0.042862)
12. feature 0 (0.037581)
13. feature 2 (0.020206)
14. feature 8 (0.016274)
15. feature 6 (0.016247)
16. feature 14 (0.013058)
17. feature 23 (0.008264)
18. feature 5 (0.006849)
19. feature 4 (0.005578)
20. feature 12 (0.004800)
21. feature 7 (0.001551)
22. feature 13 (0.000468)
23. feature 22 (0.000194)
24. feature 19 (0.000000)


As we have seen that the best accuracy was achieved by using ADA Boosting on Decision tree regressor. 